In [1]:
rm(list= ls())
ls()


character(0)

# modle glycoprotein genes

In [3]:
library(rlang)
library(caret)
library(randomForest)
library(e1071)
library(dplyr)
library(LogicReg )
library(stabs)
library(mboost)
library(pROC)

载入需要的程辑包：ggplot2

载入需要的程辑包：lattice

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


载入程辑包：‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin



载入程辑包：‘dplyr’


The following object is masked from ‘package:randomForest’:

    combine


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


载入需要的程辑包：survival


载入程辑包：‘survival’


The following object is masked from ‘package:caret’:

    cluster


载入需要的程辑包：parallel


载入程辑包：‘mboost’


The following object is masked from ‘package:ggplot2’:

    %+%


Type 'citation("pROC")' for a citation.


载入程辑包：‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




In [3]:

genes<-c('Thbs1','Ltbp2','Postn',
         'Pcolce','Mfap4','Fn1','Fbln1','Igfbp7',
         'Mfge8',
         'Spp1','Igfbp4','Mgp')
setwd("./data/modle/")

In [5]:
gene_expression <-t( read.table("./data/modle/count.matrix.txt"))
label <- read.csv("./data/modle/label.csv")


In [6]:
table(colnames(gene_expression) %in% genes)
gene_expression<-gene_expression[,colnames(gene_expression) %in% genes]
dim(gene_expression)


FALSE  TRUE 
    5    12 

[1] 4537   12

In [7]:
# 将标签向量转换为因子类型
labels <- as.factor(label$ZsGreen1_type)

In [9]:
set.seed(123)
train_indices <- createDataPartition(labels, p = 0.8, list = FALSE) #0.75

In [10]:
train_data <- gene_expression[train_indices,]
train_labels <- as.factor(label[train_indices,]$ZsGreen1_type)
test_data <- gene_expression[-train_indices, ]
test_labels <- as.factor(label[-train_indices,]$ZsGreen1_type)
dim(train_data)
dim(test_data)

[1] 3630   12

[1] 907  12

In [11]:
# train_data
data<-as.data.frame(train_data)
data$type<-train_labels
data$type2<-ifelse(data$type=="ZsGreen1+","Y","N")
data<-data[,c(1:12,14)]
# head(data)
# test data
test_data<-as.data.frame(test_data)
head(test_data)

test_label<-as.data.frame(test_labels)
test_label$type2<-ifelse(test_label$test_labels=="ZsGreen1+","Y","N")
# head(test_label)

,Thbs1,Ltbp2,Postn,Pcolce,Mfap4,Fn1,Fbln1,Igfbp7,Mfge8,Spp1,Igfbp4,Mgp
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
GSM4705592_RPS003_ACACCCTTCTAAGCCA,0,1,1,2,0,10,0,71,14,0,4,711
GSM4705592_RPS003_ACACTGACACAGGCCT,0,1,2,20,25,5,0,142,2,0,9,398
GSM4705592_RPS003_CGTAGCGAGGCATTGG,1,0,3,9,14,0,6,33,0,0,43,13
GSM4705592_RPS003_CTAGCCTGTAGCGTGA,0,1,0,4,1,0,5,18,0,0,3,32
GSM4705592_RPS003_GCAGTTAGTGCAGTAG,7,0,0,3,0,4,0,125,17,0,6,135
GSM4705592_RPS003_GCTGGGTGTTTGGGCC,0,0,11,2,8,3,0,125,9,0,17,577


In [12]:

trControl <- trainControl(method="repeatedcv", number=10, repeats=3)

model <- train(type2 ~ .,data = data, method = "rf",trControl = trControl)



In [13]:
model

Random Forest 

3630 samples
  12 predictor
   2 classes: 'N', 'Y' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 3267, 3267, 3266, 3267, 3267, 3268, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
   2    0.8321346  0.6595193
   7    0.8283780  0.6510369
  12    0.8248847  0.6436955

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [14]:

predictions1<- predict(model, newdata = test_data, type = "prob")

# 评估模型性能
predictions2<- predict(model, newdata = test_data)
confusionMatrix(predictions2,as.factor(test_label$type2))

Confusion Matrix and Statistics

          Reference
Prediction   N   Y
         N 335  69
         Y  66 437
                                          
               Accuracy : 0.8512          
                 95% CI : (0.8263, 0.8737)
    No Information Rate : 0.5579          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.6985          
                                          
 Mcnemar's Test P-Value : 0.8633          
                                          
            Sensitivity : 0.8354          
            Specificity : 0.8636          
         Pos Pred Value : 0.8292          
         Neg Pred Value : 0.8688          
             Prevalence : 0.4421          
         Detection Rate : 0.3693          
   Detection Prevalence : 0.4454          
      Balanced Accuracy : 0.8495          
                                          
       'Positive' Class : N               
                              

In [18]:
roc_obj <- roc(as.factor(test_label$type2), predictions1[,"Y"])
auc_value <- auc(roc_obj)
print(auc_value)
pdf("trained_model_gl_rf_roc_GSE155513_ldlr_test.pdf",width =6,height = 5 )
p<-plot(roc_obj, main = paste0("rf model gl ROC Curve","\n","Area under the curve:",round(auc_value,4)), xlab = "False Positive Rate", ylab = "True Positive Rate")
dev.off()

# saveRDS(model, "./data/modle/trained_model_gl_rf_roc.rds")

Setting levels: control = N, case = Y

Setting direction: controls < cases



Area under the curve: 0.9285


In [ ]:
0.9285

# 10X apoe test


In [19]:
# 保存模型


# 加载模型
model <- readRDS("./data/modle/trained_model_gl_rf_roc.rds")


In [23]:
obj<-get(load("./data/modle/Apoe_fib.RData"))

In [24]:
count<-t(as.matrix(obj@assays$RNA@counts))[,genes]
label<-obj@meta.data[,c("cells","ZsGreen1_type")]

In [25]:
test_data<-as.data.frame(count)
# head(test_data)

test_label<-label
test_label$type<-ifelse(test_label$ZsGreen1_type=="ZsGreen1+","Y","N")
# head(test_label)

In [26]:

predictions1<- predict(model, newdata = test_data, type = "prob")

# 评估模型性能
predictions2<- predict(model, newdata = test_data)
confusionMatrix(predictions2,as.factor(test_label$type))

Confusion Matrix and Statistics

          Reference
Prediction    N    Y
         N 1143  146
         Y  125  600
                                          
               Accuracy : 0.8654          
                 95% CI : (0.8498, 0.8801)
    No Information Rate : 0.6296          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.7098          
                                          
 Mcnemar's Test P-Value : 0.2244          
                                          
            Sensitivity : 0.9014          
            Specificity : 0.8043          
         Pos Pred Value : 0.8867          
         Neg Pred Value : 0.8276          
             Prevalence : 0.6296          
         Detection Rate : 0.5675          
   Detection Prevalence : 0.6400          
      Balanced Accuracy : 0.8529          
                                          
       'Positive' Class : N               
                        

In [27]:

roc_obj <- roc(as.factor(test_label$type), predictions1[,"Y"])
auc_value <- auc(roc_obj)
print(auc_value)
# pdf("trained_model_gl_rf_roc_GSE155513_apoe_test.pdf",width =6,height = 5 )
# p<-plot(roc_obj, main = paste0("rf model gl ROC Curve","\n","Area under the curve:",round(auc_value,4)), xlab = "False Positive Rate", ylab = "True Positive Rate")
# dev.off()

# saveRDS(model, "trained_model_gl_rf_roc.rds")

Setting levels: control = N, case = Y

Setting direction: controls < cases



Area under the curve: 0.9408


# 10X human test

In [3]:
library(Seurat)
library(RCurl)
library(cowplot)

library(tidyverse)
library(ggplot2)
library(Matrix)
library(scales)


library(dplyr)

library(patchwork)

library(rlang)
library(caret,lib.loc = "/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1")

library(randomForest,lib.loc = "/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1")

library(e1071)
library(dplyr)
library(LogicReg,lib.loc ="/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1" )
library(stabs,lib.loc="/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1" )
library(mboost,lib.loc="/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1")
library(pROC,lib.loc="/DATA/User/hemingmin/packages/R/x86_64-conda-linux-gnu-library/4.1")

载入需要的程辑包：SeuratObject

载入需要的程辑包：sp


载入程辑包：‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::complete()  masks RCurl::complete()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ lubridate::stamp() masks cowplot::stamp()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

载入程辑包：‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



载入程辑包：‘scales’


The following object is masked from ‘package:purrr’:

    discard


The following object is masked from ‘package:readr’:

    co

In [4]:
model <- readRDS("./data/modle/trained_model_gl_rf_roc.rds")

In [5]:
obj<-readRDS(file = "./data/modle/human_anno.rds")

In [7]:
fib<-subset(obj,idents=c("Fib"))
# fib
Idents(fib)<-fib$type


In [8]:
table(fib$type)


  AC   PA 
  49 1079 

In [9]:
# 重新scale
DefaultAssay(fib)<-"RNA"
obj<-NormalizeData(fib)
all.genes<-rownames(obj)
obj<-ScaleData(obj,features=all.genes)

Centering and scaling data matrix



In [11]:
genes<-c('THBS1','LTBP2','POSTN',
         'PCOLCE','MFAP4','FN1','FBLN1','IGFBP7',
         'MFGE8',
         'SPP1','IGFBP4','MGP')

genes_m<-c('Thbs1','Ltbp2','Postn',
         'Pcolce','Mfap4','Fn1','Fbln1','Igfbp7',
         'Mfge8',
         'Spp1','Igfbp4','Mgp')




In [12]:
counts<-t(as.matrix(obj@assays$RNA@counts[genes,]))
dim(counts)

[1] 1128   12

In [1]:
# table(colnames(counts) %in% genes)

In [14]:
colnames(counts)[colnames(counts) == 'THBS1'] <- 'Thbs1'
colnames(counts)[colnames(counts) == 'LTBP2'] <- 'Ltbp2'
colnames(counts)[colnames(counts) == 'POSTN'] <- 'Postn'
colnames(counts)[colnames(counts) == 'PCOLCE'] <- 'Pcolce'
colnames(counts)[colnames(counts) == 'MFAP4'] <- 'Mfap4'
colnames(counts)[colnames(counts) == 'FN1'] <- 'Fn1'
colnames(counts)[colnames(counts) == 'FBLN1'] <- 'Fbln1'
colnames(counts)[colnames(counts) == 'IGFBP7'] <- 'Igfbp7'
colnames(counts)[colnames(counts) == 'MFGE8'] <- 'Mfge8'
colnames(counts)[colnames(counts) == 'SPP1'] <- 'Spp1'
colnames(counts)[colnames(counts) == 'IGFBP4'] <- 'Igfbp4'
colnames(counts)[colnames(counts) == 'MGP'] <- 'Mgp'

In [2]:
# table(colnames(counts) %in% genes_m)

In [16]:
label<-obj@meta.data[,c("cells","type")]

In [17]:
test_data<-as.data.frame(counts)
# head(test_data)

test_label<-label
test_label$type<-ifelse(test_label$type=="AC","Y","N")
# head(test_label)

In [18]:

predictions1<- predict(model, newdata = test_data, type = "prob")

# 评估模型性能
predictions2<- predict(model, newdata = test_data)
confusionMatrix(predictions2,as.factor(test_label$type))

Confusion Matrix and Statistics

          Reference
Prediction   N   Y
         N 851  18
         Y 228  31
                                          
               Accuracy : 0.7819          
                 95% CI : (0.7567, 0.8057)
    No Information Rate : 0.9566          
    P-Value [Acc > NIR] : 1               
                                          
                  Kappa : 0.1383          
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.7887          
            Specificity : 0.6327          
         Pos Pred Value : 0.9793          
         Neg Pred Value : 0.1197          
             Prevalence : 0.9566          
         Detection Rate : 0.7544          
   Detection Prevalence : 0.7704          
      Balanced Accuracy : 0.7107          
                                          
       'Positive' Class : N               
                              

In [19]:

roc_obj <- roc(as.factor(test_label$type), predictions1[,"Y"])
auc_value <- auc(roc_obj)
print(auc_value)
pdf("trained_model_gl_rf_roc_GSE159677_human_test_fib.pdf",width =6,height = 5 )
p<-plot(roc_obj, main = paste0("rf model gl ROC Curve","\n","Area under the curve:",round(auc_value,4)), xlab = "False Positive Rate", ylab = "True Positive Rate")
dev.off()



Setting levels: control = N, case = Y

Setting direction: controls < cases



Area under the curve: 0.8028


png 
  2